In [ ]:
from os import listdir, getpid
from os.path import isfile, join
import pandas as pd
import numpy as np
from multiprocessing import Pool, Process
import json
import zipfile
import os
import multiprocessing
import FinanceDataReader as fdr

In [ ]:
def time_maker(time):
        # 9시 date 값 YY/MM/DD/90/mm -> YY/MM/DD/09/mm으로 변경
        if time[8] == '9':
            return time[:8] + '09' + time[9:-1]
        else:
            return time

def timeline(dir_route,time_maker):
    # 시간데이터 수정해서 다시 저장하는 함수
    directory = 'C:\\Users\\john\\work_data\\Data\\1m_date_modified' # 저장 파일 경로를 설정해 준다.

    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file_name_list = os.listdir(dir_route)[1:]
    
    for i in range(len(file_name_list)):
        print(i)
        data = pd.read_pickle(dir_route + '\\' + file_name_list[i])   
        data['date'] = data['date'].apply(time_maker)
        data.to_pickle(directory + '\\' + file_name_list [i] )

    return 'date수정완료'

timeline('C:\\Users\\john\\work_data\\Data\\1m_all_day_sum',time_maker)

In [ ]:
directory = 'C:\\Users\\john\\work_data\\Data\\1m_date_modified2' # 저장 파일 경로를 설정해 준다.
if not os.path.exists(directory):
    os.makedirs(directory)
    
file_list = os.listdir('C:\\Users\\john\\work_data\\Data\\1m_all_day_sum')[1:] # 폴터 코드별 파일 종류 추출
file_list

In [ ]:
def max_row(file_dir_route): 
    
    #파일안에 쵀대 row 값을 가진 파일명을 추출 및 최대값 
    #같은 값을가진 파일명을 담은 리스트
    #최대시간값에 맞쳐만든 분봉 데이터프레임을 출력한다.
    
    file_name_list = os.listdir(file_dir_route)[1:] # 파일안에있는 파일명 전체를 읽어 리스트에 담는다.
    
    data_len = []
    count = 0
    #코드별 디렉토리 안에있는 파일(월데이터) 개수 파악하여 date_len 리스트에 값을 넣어준다
    for i in file_name_list:
        m1_data = pd.read_pickle(file_dir_route + '\\' + i)
        hour_time = m1_data['date']
        a = hour_time.apply(lambda x : x[:10])
        b = a.unique()
        data_len.append(len(b))
        count += 1
        print(count)
        
    max_index = data_len.index(max(data_len))
    
    max_row_event_name = file_name_list[max_index]
    
    max_m1_data = pd.read_pickle(file_dir_route + '\\' + max_row_event_name)
    max_hour_time = max_m1_data['date']
    A = max_hour_time.apply(lambda x : x[:10])
    B = A.unique()
    max_row_len = len(B)
    
    # 시간 기준으로 분봉 데이터프레임을 만든다.
    time_h = B

    all_time_list = {'date':[]}
    
    for i in time_h:
        
        if i[-2:] == '09':
            for k in range(1,60):
                if k < 10 :
                    time = i + '0' + str(k)
                    all_time_list['date'].append(time)
                else:
                    time = i + str(k)
                    all_time_list['date'].append(time)
    
        elif i[-2:] == '15': 
            for k in range(32):
                if k < 10 :
                    time = i + '0' + str(k)
                    all_time_list['date'].append(time)
                else:
                    time = i + str(k)
                    all_time_list['date'].append(time)
        else:
            for k in range(60):
                if k < 10 :
                    time = i + '0' + str(k)
                    all_time_list['date'].append(time)
                else:
                    time = i + str(k)
                    all_time_list['date'].append(time)
        
    all_time_df_ = pd.DataFrame(all_time_list)
    all_time_df_['a'] = '0' #나중에 join 할때 편하게 하려고 a값을 넣어준다.
    all_time_df = all_time_df_.set_index('date')
    
    maxfilter_datalist = []
    
    for i in range(len(data_len)):
        
        if data_len[i] == max_row_len:
            maxfilter_datalist.append(file_name_list[i])
            
    
               
    return max_row_event_name , max_row_len, maxfilter_datalist , all_time_df
           #최대 로우 종목명   ,  최대 로우값,  시간필터 결과 종록 리스트 , 최대시간기준 최대 분봉 

In [ ]:
max_row_event_name , max_row_len, file_name_list , all_time_df = max_row(directory) 

In [ ]:
import FinanceDataReader as fdr

# 시작 값이 없을시 전날 종가기준으로 맞쳐주기위해서 전날 일봉 종가데이터를 수집힌다.
start_price = [] # 전날 종가를 넣어준다 만약 조회가 안될시 nan 값으로 들어간다.
for i in range(len(file_name_list)):
    print(file_name_list[i][6:12], ' ',i, ' ', file_name_list[i])
    try:
        df = fdr.DataReader(file_name_list[i][6:12],'2017-07-31','2017-07-31')
        close_price = df['Close'][0]
        start_price.append(close_price)
        print(file_name_list[i][6:12], ' ',i, ' ',close_price,' ', file_name_list[i])
    except:
        start_price.append('nan')


In [ ]:
for i in range(len(file_name_list)):

    close = start_price[i] # 최대 시간row 값을 가진 종목을 담은 리스트와 같이 순서를 맞쳐서 출력된다.
    
    if close == 'nan':
        pass
    
    else:
        ex = pd.read_pickle('C:\\Users\\john\\work_data\\Data\\1m_date_modified2\\' + file_name_list[i])

        df2  = ex.set_index('date')

        join_data = all_time_df.join(df2, how='left')

        result = join_data.drop('a',axis=1)
        result['volume'] = result['volume'].fillna('0') # volume null값들 0으로 치환
        
        if  result['volume'][0] == '0':
                # 첫번째 volume row 값이 0일시 ohlc 값이 null 값임으로 일봉 전날 종가로 값을 치환해준다.
                result['open'][0] = close
                result['high'][0] = close
                result['low'][0] = close
                result['close'][0] = close
           
        result['close'] = result.fillna(method = 'ffill')['close']   # 전 row 값의 close 값으로 치환한다.
        result['open'] = result.fillna(method = 'ffill')['close']    # 전 row 값의 close 값으로 치환한다.
        result['low'] = result.fillna(method = 'ffill')['close']     # 전 row 값의 close 값으로 치환한다.
        result['high'] = result.fillna(method = 'ffill')['close']    # 전 row 값의 close 값으로 치환한다.
   
        print(i)

        result3 = result.reset_index()
        result3.to_pickle('C:\\Users\\john\\work_data\\data\\1m_filter_data\\'+ file_name_list[i] ) # 합친데이터 출력
